In [1]:
import torch
import pandas as pd

In [2]:
class Neuron:
    def __init__(self, num_inputs, num_outputs):
        self.w = torch.randn(num_inputs, num_outputs, requires_grad=True, dtype=torch.float32)
        self.b = torch.randn(num_outputs, 1, requires_grad=True, dtype=torch.float32)

    def forward(self, inputs):
        self.out = (inputs @ self.w + self.b).relu()
        return self.out

    def train(self, inputs, outputs, epochs, learning_rate):
        for i in range(epochs):
            loss = ((outputs - self.forward(inputs))**2).flatten().sum()
            loss.backward()
            print(f"loss: {loss.data}")
            self.w.data = self.w.data - self.w.grad * learning_rate 
            self.b.data = self.b.data - self.b.grad * learning_rate 
            self.w.grad = None
            self.b.grad = None
            


In [3]:
class Layer:
    def __init__(self, num_neurons, inputs_per_neuron): 
        self.num_neurons = num_neurons
        self.w = torch.randn(num_neurons, inputs_per_neuron, requires_grad=True, dtype=torch.float32)
        self.b = torch.randn(num_neurons, 1, requires_grad=True, dtype=torch.float32)
        

    def forward(self, inputs):
        out = ((self.w @ inputs) + self.b)
        return out 
        

In [4]:
class MLP:
    def __init__(self, num_layers, num_inputs, num_neurons_per_layer, num_final_out):
        self.num_layers = num_layers
        self.layers = [Layer(num_neurons_per_layer, num_inputs)]

        for _ in range(num_layers): 
            layer = Layer(num_neurons_per_layer, num_neurons_per_layer)
            self.layers.append(layer)
        
        layer_out = Layer(num_final_out, num_neurons_per_layer)
        self.layers.append(layer_out)
        
    def forward(self, input):
        out_prev = input
        for i in range(len(self.layers)):
            if i == (len(self.layers)-1):
                out_prev = self.layers[i].forward(out_prev)
            else:
                out_prev = self.layers[i].forward(out_prev).tanh()
        return out_prev
        
    def train(self, epochs, learning_rate, x, y):
        loss_func = torch.nn.CrossEntropyLoss()
        for _ in range(epochs):
            # loss = ((y - self.forward(x))**2).flatten().sum()
            y_pred = self.forward(x).view(y.shape[0], y.shape[1])
            # print(y_pred.shape, y_pred.dtype)
            loss = loss_func(y_pred, y)
            print(f"Loss: {loss}")
            loss.backward()
            for layer in self.layers:
                layer.w.data -= learning_rate * layer.w.grad
                layer.b.data -= learning_rate * layer.b.grad
                layer.w.grad = None
                layer.b.grad = None
            
        

In [5]:

epochs = 2000
learning_rate = 0.05
num_neurons_per_layer = 16
num_layers = 2
train_split = 0.90
batch_size = int(150*train_split)

In [6]:
def datagen(batch_size):
    headers=["sepal length", "sepal width", "petal length", "petal width", "name"]
    data = pd.read_csv("iris.data", names=headers).sample(frac=1).reset_index(drop=True)

    dataset_size = len(data) # 150
    num_classifications = len(set(data.name)) # 3
    num_features = len(data.columns) - 1 # 4

    rand_seed_idx = torch.randint((dataset_size - batch_size), (1,)).item() #generates a tensor with a random integer and extracts it using .item()
    training_data = data[rand_seed_idx : rand_seed_idx+batch_size].reset_index(drop=True, inplace=False)

    diff_flowers = set(data.name)
    classification = []
    for idx in range(len(training_data['name'])):
        if training_data.name[idx] == "Iris-setosa":
            classification.append([1,0,0])
        elif training_data.name[idx] == "Iris-versicolor":
            classification.append([0,1,0])
        elif training_data.name[idx] == "Iris-virginica":
            classification.append([0,0,1])
    
    training_data['classification'] = classification
    features_list=[]
    for i in range(num_features):
        features_list.append(torch.tensor(training_data.iloc(axis=1)[i].tolist()).reshape(len(training_data.iloc(axis=1)[0].tolist()), 1)) 

    train_in = torch.cat(features_list, dim=1)
    train_out = torch.tensor(training_data['classification'].tolist(), dtype=torch.float32)
    train_in = train_in.reshape(batch_size, num_features, 1)
    train_out = train_out.reshape(batch_size, num_classifications)

    examples = train_split * dataset_size
    num_inputs = num_features
    num_outputs = num_classifications

    return train_in, train_out, num_features, num_classifications

In [7]:
train_in, train_out, num_features, num_classifications = datagen(batch_size)

In [8]:
classifier = MLP(num_layers=3, num_inputs=num_features, num_neurons_per_layer=num_neurons_per_layer, num_final_out=num_classifications)

In [11]:
out = classifier.forward(train_in).view(train_out.shape[0], train_out.shape[1])

In [14]:
out[-1], train_out[-1]

(tensor([ 3.5833, -4.9742, -4.6019], grad_fn=<SelectBackward0>),
 tensor([1., 0., 0.]))

In [10]:
classifier.train(epochs, learning_rate, train_in, train_out)

Loss: 5.053279876708984
Loss: 2.4837591648101807
Loss: 1.8253629207611084
Loss: 1.5550003051757812
Loss: 1.3637259006500244
Loss: 1.1544020175933838
Loss: 0.8570358157157898
Loss: 0.6860980987548828
Loss: 0.6182630658149719
Loss: 0.5898454785346985
Loss: 0.5749603509902954
Loss: 0.5642157196998596
Loss: 0.54957115650177
Loss: 0.5312315225601196
Loss: 0.5219758152961731
Loss: 0.5172961950302124
Loss: 0.513846218585968
Loss: 0.5110434293746948
Loss: 0.508697509765625
Loss: 0.5066901445388794
Loss: 0.5049418807029724
Loss: 0.5033968687057495
Loss: 0.5020154118537903
Loss: 0.5007680654525757
Loss: 0.4996331036090851
Loss: 0.49859392642974854
Loss: 0.49763765931129456
Loss: 0.49675434827804565
Loss: 0.49593567848205566
Loss: 0.49517500400543213
Loss: 0.4944666624069214
Loss: 0.49380573630332947
Loss: 0.4931878447532654
Loss: 0.49260902404785156
Loss: 0.49206554889678955
Loss: 0.4915541410446167
Loss: 0.4910717010498047
Loss: 0.4906151592731476
Loss: 0.49018198251724243
Loss: 0.4897695481777